# google-covid-19-mobility-data-process-london

In this notebook I will refine the process from `google-covid-19-mobility-data-process-v1` in order to get the highest resolution data possible for a map of the boroughs of London.

In [2]:
import pandas as pd
import requests
import simplejson as json
import numpy as np

---

## Load a reduced CSV containing just the Greater London entries

In [5]:
londonDf = pd.read_csv("./output-data/london.csv")

In [6]:
londonDf.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-15,-5.0,-9.0,-12.0,-11,NaN,NaN
1,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-16,-1.0,-21.0,-23.0,-13,NaN,NaN
2,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-17,-3.0,-2.0,4.0,-1,-4.0,NaN
3,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-18,-2.0,-2.0,-1.0,-2,-2.0,NaN
4,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-19,-7.0,-4.0,5.0,0,-4.0,NaN


---

## Get the lat long coordinates for each unique `place_id`

In [7]:
uniquePlaceIdsDf = londonDf[["place_id"]].drop_duplicates()

In [8]:
uniquePlaceIdsDf.head()

,place_id
0,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ
462,ChIJxwN8mDUFdkgRoGfsoi2uDgQ
924,ChIJkcYYXrKl2EcR0Ge-JsaFEQQ
1386,ChIJT29HcDkXdkgR4Gfsoi2uDgQ
1848,ChIJIfhvfFqu2EcR8Ge-JsaFEQQ


Access the google maps api to get coordinates for each `place_id`

In [9]:
with open('./secrets/googleapikey.txt', 'r') as f:
    key = f.read()

In [10]:
def get_lat_long(place_id):
    try:
        API_KEY = key.rstrip("\n")
        url = "https://maps.googleapis.com/maps/api/place/details/json?place_id=" + str(place_id) + "&key=" + str(API_KEY) + "&fields=geometry"
        
        response = (requests.get(url).text)
        response_json = json.loads(response)
        
        if "result" in response_json:
            result = response_json["result"]
            if "geometry" in result:
                geometry = result["geometry"]
                if "location" in geometry:
                    location = geometry["location"]
                    return location["lat"], location["lng"]
                else:
                    return None, None
            else:
                return None, None
        else:
            return None, None
    
    except Exception as e:
        raise e

In [11]:
uniquePlaceIdsDf.loc[:, "lat"], uniquePlaceIdsDf.loc[:, "lng"] = zip(*uniquePlaceIdsDf['place_id'].map(get_lat_long))

In [12]:
uniquePlaceIdsDf.head()

,place_id,lat,lng
0,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,51.512344,-0.090985
462,ChIJxwN8mDUFdkgRoGfsoi2uDgQ,51.514534,-0.159539
924,ChIJkcYYXrKl2EcR0Ge-JsaFEQQ,51.554067,0.134017
1386,ChIJT29HcDkXdkgR4Gfsoi2uDgQ,51.604967,-0.207629
1848,ChIJIfhvfFqu2EcR8Ge-JsaFEQQ,51.451902,0.117179


I'll save these coordinates as a CSV file for later use.

In [13]:
uniquePlaceIdsDf.to_csv("./output-data/lonodon-boroughs-lat-lng.csv", index=False)

In [14]:
len(uniquePlaceIdsDf)

33

---

## Merge the coordinates with the original `sub_region_2` data frame

In [7]:
uniquePlaceIdsDf = pd.read_csv("./output-data/usa-sub-region-2-lat-lng.csv", low_memory=False)

In [15]:
londonMergeDf = pd.merge(londonDf, uniquePlaceIdsDf, on='place_id', how='outer')

In [16]:
londonMergeDf.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,lat,lng
0,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-15,-5.0,-9.0,-12.0,-11,NaN,NaN,51.512344,-0.090985
1,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-16,-1.0,-21.0,-23.0,-13,NaN,NaN,51.512344,-0.090985
2,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-17,-3.0,-2.0,4.0,-1,-4.0,NaN,51.512344,-0.090985
3,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-18,-2.0,-2.0,-1.0,-2,-2.0,NaN,51.512344,-0.090985
4,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-19,-7.0,-4.0,5.0,0,-4.0,NaN,51.512344,-0.090985


---

## Calculate 7 day rolling averages for each location

In [17]:
def add_rolling_average(df):
    df.loc[:, "retail-average"] = df["retail_and_recreation_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    df.loc[:, "grocery-average"] = df["grocery_and_pharmacy_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    df.loc[:, "parks-average"] = df["parks_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    df.loc[:, "transit-average"] = df["transit_stations_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    df.loc[:, "workplace-average"] = df["workplaces_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    df.loc[:, "residential-average"] = df["residential_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    return df

In [18]:
londonAverageDf = londonMergeDf.groupby("place_id").apply(add_rolling_average)

In [20]:
londonAverageDf[5:10]

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,...,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,lat,lng,retail-average,grocery-average,parks-average,transit-average,workplace-average,residential-average
5,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-20,-7.0,...,-3.0,NaN,51.512344,-0.090985,-1.428571,-1.571429,1.285714,-3.571429,NaN,NaN
6,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-21,-3.0,...,-4.0,NaN,51.512344,-0.090985,-2.000000,-3.000000,-1.142857,-3.571429,NaN,NaN
7,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-22,4.0,...,NaN,NaN,51.512344,-0.090985,-1.857143,-2.571429,-1.714286,-3.428571,NaN,NaN
8,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-23,8.0,...,NaN,NaN,51.512344,-0.090985,-0.857143,-2.428571,-2.285714,-3.857143,NaN,NaN
9,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-24,-7.0,...,2.0,NaN,51.512344,-0.090985,0.142857,-2.000000,-2.000000,-3.571429,NaN,NaN


---

## Round the average figures to 1 decimal place for a smaller final file size

In [21]:
londonRoundedDf = londonAverageDf.round({
    'retail-average': 1,
    'grocery-average': 1,
    'parks-average': 1,
    'transit-average': 1,
    'workplace-average': 1,
    'residential-average': 1,
})

In [22]:
londonRoundedDf[5:10]

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,...,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,lat,lng,retail-average,grocery-average,parks-average,transit-average,workplace-average,residential-average
5,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-20,-7.0,...,-3.0,NaN,51.512344,-0.090985,-1.4,-1.6,1.3,-3.6,NaN,NaN
6,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-21,-3.0,...,-4.0,NaN,51.512344,-0.090985,-2.0,-3.0,-1.1,-3.6,NaN,NaN
7,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-22,4.0,...,NaN,NaN,51.512344,-0.090985,-1.9,-2.6,-1.7,-3.4,NaN,NaN
8,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-23,8.0,...,NaN,NaN,51.512344,-0.090985,-0.9,-2.4,-2.3,-3.9,NaN,NaN
9,GB,United Kingdom,Greater London,City of London,NaN,GB-LND,NaN,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,2020-02-24,-7.0,...,2.0,NaN,51.512344,-0.090985,0.1,-2.0,-2.0,-3.6,NaN,NaN


---

## Convert the data into a python dictionary so it can be exported as json

In [28]:
dates = ["2020-02-15","2020-02-16","2020-02-17","2020-02-18","2020-02-19","2020-02-20","2020-02-21","2020-02-22","2020-02-23","2020-02-24","2020-02-25","2020-02-26","2020-02-27","2020-02-28","2020-02-29","2020-03-01","2020-03-02","2020-03-03","2020-03-04","2020-03-05","2020-03-06","2020-03-07","2020-03-08","2020-03-09","2020-03-10","2020-03-11","2020-03-12","2020-03-13","2020-03-14","2020-03-15","2020-03-16","2020-03-17","2020-03-18","2020-03-19","2020-03-20","2020-03-21","2020-03-22","2020-03-23","2020-03-24","2020-03-25","2020-03-26","2020-03-27","2020-03-28","2020-03-29","2020-03-30","2020-03-31","2020-04-01","2020-04-02","2020-04-03","2020-04-04","2020-04-05","2020-04-06","2020-04-07","2020-04-08","2020-04-09","2020-04-10","2020-04-11","2020-04-12","2020-04-13","2020-04-14","2020-04-15","2020-04-16","2020-04-17","2020-04-18","2020-04-19","2020-04-20","2020-04-21","2020-04-22","2020-04-23","2020-04-24","2020-04-25","2020-04-26","2020-04-27","2020-04-28","2020-04-29","2020-04-30","2020-05-01","2020-05-02","2020-05-03","2020-05-04","2020-05-05","2020-05-06","2020-05-07","2020-05-08","2020-05-09","2020-05-10","2020-05-11","2020-05-12","2020-05-13","2020-05-14","2020-05-15","2020-05-16","2020-05-17","2020-05-18","2020-05-19","2020-05-20","2020-05-21","2020-05-22","2020-05-23","2020-05-24","2020-05-25","2020-05-26","2020-05-27","2020-05-28","2020-05-29","2020-05-30","2020-05-31","2020-06-01","2020-06-02","2020-06-03","2020-06-04","2020-06-05","2020-06-06","2020-06-07","2020-06-08","2020-06-09","2020-06-10","2020-06-11","2020-06-12","2020-06-13","2020-06-14","2020-06-15","2020-06-16","2020-06-17","2020-06-18","2020-06-19","2020-06-20","2020-06-21","2020-06-22","2020-06-23","2020-06-24","2020-06-25","2020-06-26","2020-06-27","2020-06-28","2020-06-29","2020-06-30","2020-07-01","2020-07-02","2020-07-03","2020-07-04","2020-07-05","2020-07-06","2020-07-07","2020-07-08","2020-07-09","2020-07-10","2020-07-11","2020-07-12","2020-07-13","2020-07-14","2020-07-15","2020-07-16","2020-07-17","2020-07-18","2020-07-19","2020-07-20","2020-07-21","2020-07-22","2020-07-23","2020-07-24","2020-07-25","2020-07-26","2020-07-27","2020-07-28","2020-07-29","2020-07-30","2020-07-31","2020-08-01","2020-08-02","2020-08-03","2020-08-04","2020-08-05","2020-08-06","2020-08-07","2020-08-08","2020-08-09","2020-08-10","2020-08-11","2020-08-12","2020-08-13","2020-08-14","2020-08-15","2020-08-16","2020-08-17","2020-08-18","2020-08-19","2020-08-20","2020-08-21","2020-08-22","2020-08-23","2020-08-24","2020-08-25","2020-08-26","2020-08-27","2020-08-28","2020-08-29","2020-08-30","2020-08-31","2020-09-01","2020-09-02","2020-09-03","2020-09-04","2020-09-05","2020-09-06","2020-09-07","2020-09-08","2020-09-09","2020-09-10","2020-09-11","2020-09-12","2020-09-13","2020-09-14","2020-09-15","2020-09-16","2020-09-17","2020-09-18","2020-09-19","2020-09-20","2020-09-21","2020-09-22","2020-09-23","2020-09-24","2020-09-25","2020-09-26","2020-09-27","2020-09-28","2020-09-29","2020-09-30","2020-10-01","2020-10-02","2020-10-03","2020-10-04","2020-10-05","2020-10-06","2020-10-07","2020-10-08","2020-10-09","2020-10-10","2020-10-11","2020-10-12","2020-10-13","2020-10-14","2020-10-15","2020-10-16","2020-10-17","2020-10-18","2020-10-19","2020-10-20","2020-10-21","2020-10-22","2020-10-23","2020-10-24","2020-10-25","2020-10-26","2020-10-27","2020-10-28","2020-10-29","2020-10-30","2020-10-31","2020-11-01","2020-11-02","2020-11-03","2020-11-04","2020-11-05","2020-11-06","2020-11-07","2020-11-08","2020-11-09","2020-11-10","2020-11-11","2020-11-12","2020-11-13","2020-11-14","2020-11-15","2020-11-16","2020-11-17","2020-11-18","2020-11-19","2020-11-20","2020-11-21","2020-11-22","2020-11-23","2020-11-24","2020-11-25","2020-11-26","2020-11-27","2020-11-28","2020-11-29","2020-11-30","2020-12-01","2020-12-02","2020-12-03","2020-12-04","2020-12-05","2020-12-06","2020-12-07","2020-12-08","2020-12-09","2020-12-10","2020-12-11","2020-12-12","2020-12-13","2020-12-14","2020-12-15","2020-12-16","2020-12-17","2020-12-18","2020-12-19","2020-12-20","2020-12-21","2020-12-22","2020-12-23","2020-12-24","2020-12-25","2020-12-26","2020-12-27","2020-12-28","2020-12-29","2020-12-30","2020-12-31","2021-01-01","2021-01-02","2021-01-03","2021-01-04","2021-01-05","2021-01-06","2021-01-07","2021-01-08","2021-01-09","2021-01-10","2021-01-11","2021-01-12","2021-01-13","2021-01-14","2021-01-15","2021-01-16","2021-01-17","2021-01-18","2021-01-19","2021-01-20","2021-01-21","2021-01-22","2021-01-23","2021-01-24","2021-01-25","2021-01-26","2021-01-27","2021-01-28","2021-01-29","2021-01-30","2021-01-31","2021-02-01","2021-02-02","2021-02-03","2021-02-04","2021-02-05","2021-02-06","2021-02-07","2021-02-08","2021-02-09","2021-02-10","2021-02-11","2021-02-12","2021-02-13","2021-02-14","2021-02-15","2021-02-16","2021-02-17","2021-02-18","2021-02-19","2021-02-20","2021-02-21","2021-02-22","2021-02-23","2021-02-24","2021-02-25","2021-02-26","2021-02-27","2021-02-28","2021-03-01","2021-03-02","2021-03-03","2021-03-04","2021-03-05","2021-03-06","2021-03-07","2021-03-08","2021-03-09","2021-03-10","2021-03-11","2021-03-12","2021-03-13","2021-03-14","2021-03-15","2021-03-16","2021-03-17","2021-03-18","2021-03-19","2021-03-20","2021-03-21","2021-03-22","2021-03-23","2021-03-24","2021-03-25","2021-03-26","2021-03-27","2021-03-28","2021-03-29","2021-03-30","2021-03-31","2021-04-01","2021-04-02","2021-04-03","2021-04-04","2021-04-05","2021-04-06","2021-04-07","2021-04-08","2021-04-09","2021-04-10","2021-04-11","2021-04-12","2021-04-13","2021-04-14","2021-04-15","2021-04-16","2021-04-17","2021-04-18","2021-04-19","2021-04-20","2021-04-21","2021-04-22","2021-04-23","2021-04-24","2021-04-25","2021-04-26","2021-04-27","2021-04-28","2021-04-29","2021-04-30","2021-05-01","2021-05-02","2021-05-03","2021-05-04","2021-05-05","2021-05-06","2021-05-07","2021-05-08","2021-05-09","2021-05-10","2021-05-11","2021-05-12","2021-05-13","2021-05-14","2021-05-15","2021-05-16","2021-05-17","2021-05-18","2021-05-19","2021-05-20","2021-05-21"]

In [29]:
def create_list_for_json(df):
    outputList = []
    listOfPlaceIds = df["place_id"].drop_duplicates().to_list()
    groupByPlaceId = df.groupby("place_id")
    
    for place_id in listOfPlaceIds:
        thisDf = groupByPlaceId.get_group(place_id)
        testList = thisDf["parks-average"].to_list()
        
        # Some of the parks columns contain all NaNs, we'll skip these
        if np.isnan(testList).all():
            continue
        
        myDict = {}
        myDict['lng'] = thisDf.iloc[0]["lng"]
        myDict["lat"] =  thisDf.iloc[0]["lat"]

        dateIndexDf = thisDf.set_index("date")
        
        # RETAIL
        
        retailList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["retail-average"])
            except:
                thisDate = np.nan
            
            retailList.append(thisDate)
        
        myDict["retail"] = retailList
        
        # GROCERY
        
        groceryList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["grocery-average"])
            except:
                thisDate = np.nan
            
            groceryList.append(thisDate)
        
        myDict["grocery"] = groceryList
        
        # PARKS
        
        parksList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["parks-average"])
            except:
                thisDate = np.nan
            
            parksList.append(thisDate)
        
        myDict["parks"] = parksList
        
        # TRANSIT
        
        transitList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["transit-average"])
            except:
                thisDate = np.nan
            
            transitList.append(thisDate)
        
        myDict["transit"] = transitList
        
        # WORKPLACE
        
        workplaceList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["workplace-average"])
            except:
                thisDate = np.nan
            
            workplaceList.append(thisDate)
        
        myDict["workplace"] = workplaceList
        
        # Residential
        
        residentialList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["residential-average"])
            except:
                thisDate = np.nan
            
            residentialList.append(thisDate)
        
        myDict["residential"] = residentialList
        

        outputList.append(myDict)
        
    return outputList

In [30]:
londonList = create_list_for_json(londonRoundedDf)

In [31]:
len(londonList)

33

In [32]:
with open("./public/data/london-all.json", "w") as outfile: 
    json.dump(londonList, outfile, ignore_nan=True)